In [1]:
import requests
from datetime import datetime
import time
import pandas as pd

# 2.3 Request Module
- https://docs.github.com/en/rest/repos/repos?apiVersion=2022-11-28

In [1]:
## requests.get(), response.status_code
import requests
response= requests.get('https://api.github.com/repositories',
                      headers={'Accept':'application/vnd.github+json'})
print(response.status_code)

200


In [12]:
## response.headers
print(response.encoding)
print(response.headers['Content-Type'])
print(response.headers['server'])
print(response.headers['Access-Control-Expose-Headers'])
print(response.headers)

utf-8
application/json; charset=utf-8
GitHub.com
ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset
{'Server': 'GitHub.com', 'Date': 'Tue, 30 Jan 2024 10:23:37 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"3c54a93639294e719c717ee9fb29aa31617be2a77aed1b49a769e1b1c9dd5ef5"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/repositories?since=369>; rel="next", <https://api.github.com/repositories{?since}>; rel="first"', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-Rat

In [21]:
## response를 json 객체로 변환하기
import json
print(len(response.json())) # 100: 100개의 데이터가 있음, response.json()
print(json.dumps(response.json()[0], indent=2)[:200])

100
{
  "id": 1,
  "node_id": "MDEwOlJlcG9zaXRvcnkx",
  "name": "grit",
  "full_name": "mojombo/grit",
  "private": false,
  "owner": {
    "login": "mojombo",
    "id": 1,
    "node_id": "MDQ6VXNlcjE=",



In [22]:
## query 매개변수를 이용하여 필요한 정보 search
response=requests.get('https://api.github.com/repositories',
                      headers={'Accept':'application/vnd.github+json'},
                     params= {'q':'data+language:python'})
print(response.status_code)

200


## 레포지토리의 댓글 모니터링

In [32]:
response= requests.get('https://api.github.com/repos/pytorch/pytorch/issues/comments')
print(response.status_code)
print(len(response.json()))
print(response.json()[29])
## 왜 30개의 댓글만 가져온 것일까?

200
30
{'url': 'https://api.github.com/repos/pytorch/pytorch/issues/comments/246085619', 'html_url': 'https://github.com/pytorch/pytorch/issues/20#issuecomment-246085619', 'issue_url': 'https://api.github.com/repos/pytorch/pytorch/issues/20', 'id': 246085619, 'node_id': 'MDEyOklzc3VlQ29tbWVudDI0NjA4NTYxOQ==', 'user': {'login': 'apaszke', 'id': 4583066, 'node_id': 'MDQ6VXNlcjQ1ODMwNjY=', 'avatar_url': 'https://avatars.githubusercontent.com/u/4583066?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/apaszke', 'html_url': 'https://github.com/apaszke', 'followers_url': 'https://api.github.com/users/apaszke/followers', 'following_url': 'https://api.github.com/users/apaszke/following{/other_user}', 'gists_url': 'https://api.github.com/users/apaszke/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/apaszke/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/apaszke/subscriptions', 'organizations_url': 'https://api.github.com/users/apaszke/orgs'

In [28]:
## 페이지 매기기
response.links
# 다음 페이지가 존재했음. 1페이지당 30개씩만 출력

{'next': {'url': 'https://api.github.com/repositories/65600975/issues/comments?page=2',
  'rel': 'next'},
 'last': {'url': 'https://api.github.com/repositories/65600975/issues/comments?page=1000',
  'rel': 'last'}}

### function: get_all_pages(url, params, headers)

In [34]:
def get_all_pages(url, params=None, headers=None):
    output_json=[]
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        output_json = response.json() # reponse -> list( dict{}, dict{},...)
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if next_url is not None:
                output_json+=get_all_pages(next_url, params, headers) # 재귀 함수를 이용하여 다음 페이지가 있을 때까지 무한 반복시킴
    return output_json

In [57]:
url="https://api.github.com/repos/pytorch/pytorch/issues/comments"
params={"since":"2022-01-31T00:00:00Z",
       "sorted":"created", "direction":"desc"}
headers={'Accept':'application/vnd.github+json'}

start=datetime.now()
output=get_all_pages(url,params,headers)
lapse=  datetime.now()-start
print("경과시간: ", lapse)
# print("output",output[:2])
df=pd.DataFrame(output)
display(df.sample(1))
print("추출된 댓글의 수",df['body'].count())
display(df[['id','created_at','body']].sample(1))

경과시간:  0:00:02.041605


,url,html_url,issue_url,id,node_id,user,created_at,updated_at,author_association,body,reactions,performed_via_github_app
12,https://api.github.com/repos/pytorch/pytorch/i...,https://github.com/pytorch/pytorch/pull/115539...,https://api.github.com/repos/pytorch/pytorch/i...,1916501942,IC_kwDOA-j9z85yO3-2,"{'login': 'pytorchmergebot', 'id': 97764156, '...",2024-01-30T10:10:58Z,2024-01-30T10:10:58Z,COLLABORATOR,@pytorchbot started a rebase job onto [refs/re...,{'url': 'https://api.github.com/repos/pytorch/...,None


추출된 댓글의 수 60


,id,created_at,body
25,1916384722,2024-01-30T09:12:04Z,@pytorchbot merge


In [52]:

before=datetime.now()
time.sleep(5)
lapse=datetime.now()-before

In [54]:
print(lapse)

0:00:05.000577


In [60]:
## 속도제한 해결: 
# X-Ratelimit-Limit
# X-Ratelimit-Remaining
# X-Ratelimit-Reset

url="https://api.github.com/repos/pytorch/pytorch/issues/comments"
response=requests.get(url)

print('X-Ratelimit-Limit: ', response.headers['X-Ratelimit-Limit'])
print('X-Ratelimit-Remaining: ', response.headers['X-Ratelimit-Remaining'])

print('Rate Limits reset at: ', datetime.fromtimestamp(int(response.headers['X-Ratelimit-Reset'])).strftime('%c'))

X-Ratelimit-Limit:  60
X-Ratelimit-Remaining:  58
Rate Limits reset at:  Tue Jan 30 22:42:58 2024


### function handle_rate_limits(response) return boolean

In [2]:
def handle_rate_limits(response):
    now=datetime.now()
    reset_time = datetime.fromtimestamp(int(response.headers['X-RateLimit-Reset'])) # api 호출 가능 개수가 리셋되는 시간
    remaining_requests= response.headers['X-Ratelimit-Remaining'] # 남은 호출 가능 횟수
    remaining_time= (reset_time-now).total_seconds() # 리셋까지 남은 시간
    intervals= remaining_time/(1.0 + int(remaining_requests)) # 남은 시간 / 남은 호출+1 => 매 호출이 일어날 시간
    print('Sleeping for', intervals)
    time.sleep(intervals)
    return True

In [14]:
## Retry Strategy
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(total=5, status_forcelist=[500,503,504], backoff_factor=1)
# status_forcelist: 어떠한 상태일 경우에 retry를 시도하는지를 지정
# backoff_factor: 서버에 대한 호출 시간을 기하급수적으로 늘려서 지연시키기

retry_adapter=HTTPAdapter(max_retries=retry_strategy)

http=requests.Session()
http.mount("https://", retry_adapter)
http.mount("http://", retry_adapter)

url='https://api.github.com/search/repositories'
params={'q':'data+language:python'}

response= http.get(url, params=params)

for item in response.json()['items'][:5]:
    print(item['name'])

awesome
kaitai_struct
data-structures-and-algorithms
Python-Roadmap
Sports_Data_Reference


In [16]:
print(len(response.json()['items']))
print(type(response.json()['items']))

30
<class 'list'>


## Total Process

In [19]:
# 1. Prepare retry strategy
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

retry_strategy = Retry(
    total=5,
    status_forcelist=[500, 503, 504],
    backoff_factor=1
)

retry_adapter = HTTPAdapter(max_retries=retry_strategy)

http = requests.Session()
http.mount("https://", retry_adapter)
http.mount("http://", retry_adapter)

# 2. Get all pages
def get_all_pages(url, param=None, header=None):
    output_json = []
    response = http.get(url, params=param, headers=header)
    if response.status_code == 200:
        output_json = response.json()
        if 'next' in response.links:
            next_url = response.links['next']['url']
            if (next_url is not None) and (handle_rate_limits(response)):  # 3. Handle limits
                output_json += get_all_pages(next_url, param, header)
    return output_json


In [ ]:
out = get_all_pages("https://api.github.com/repos/pytorch/pytorch/issues/comments", param={'since': '2023-12-01T00:00:01Z'})
df = pd.DataFrame(out)
print(df)

# 2.4 Twitter and tweepy

- API KEY: dYVvLwbQUZnpfsjovN12mji3v
- API KEY SECRET: jW7M7XIrPgJkQQOk7ZuEkFOOR2VceSsBMWJzBEoLRQ2GCppUQI

!pip install tweepy

In [25]:
import tweepy
app_api_key='dYVvLwbQUZnpfsjovN12mji3v'
app_api_secret_key='jW7M7XIrPgJkQQOk7ZuEkFOOR2VceSsBMWJzBEoLRQ2GCppUQI'
## authentification
auth=tweepy.AppAuthHandler(app_api_key, app_api_secret_key)
api=tweepy.API(auth)

print("API Host", api.host)
print("API Version", api.api_root)

ModuleNotFoundError: No module named 'tweepy'